# Внешние отраслевые источники

In [1]:
import re
import time
import pickle
from tqdm import tqdm

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import html2text
from datetime import datetime
from pprint import pprint


def get_soup(link):
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }
    soup = None
    response = requests.get(link, headers=HEADERS)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Ошибка запроса: {response.status_code}")
    return soup


def convert_to_text(soup):
    html_converter = html2text.HTML2Text()
    html_converter.ignore_links = True
    html_converter.body_width = 0
    markdown_text = html_converter.handle(str(soup))
    return markdown_text


def convert_date(date_str):
    months = {
        'январь': 1, 'февраль': 2, 'март': 3, 'апрель': 4,
        'май': 5, 'июнь': 6, 'июль': 7, 'август': 8,
        'сентябрь': 9, 'октябрь': 10, 'ноябрь': 11, 'декабрь': 12
    }
    month_str, year_str = date_str.lower().split()
    month = months.get(month_str)
    year = int(year_str)

    return datetime(year, month, 1)

Результатом пасинга с каждого сайта будет аля jsonlines формата:

## Формат данных 

Результат представляет собой массив словарей, где каждый словарь содержит информацию об одном аналитическом материале. Каждый словарь имеет следующие ключи:

### Ключи словаря:
1. **`link`** (`str`):
   - Ссылка на страницу с аналитическим материалом. Может быть как полная, так и относительная
   - Пример: `"/news/12345"`.

2. **`date`** (`str`):
   - Дата публикации материала.
   - Пример: `"2023-10-15"`.

3. **`name`** (`str`):
   - Название материала на русском языке.
   - Пример: `"Анализ рынка ИТ в 2023 году"`.

4. **`name_en`** (`str`):
   - Название материала на английском языке, извлеченное из ссылки.
   - Пример: `"it-market-analysis-2023"`.

5. **`tags`** (`list` of `str`):
   - Список тегов, связанных с материалом.
   - Пример: `["ИТ", "аналитика", "2023"]`.

6. **`pdf_links`** (`list` of `str`):
   - Список ссылок на PDF-файлы, связанные с материалом.
   - Пример: `["https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"]`.

7. **`text`** (`str`):
   - Основной текст материала из статьи на сайте (в pdf больше информации), очищенный от лишних элементов (контакты, реклама, скрипты и т.д.).
   - Пример: `"В 2023 году рынок ИТ показал рост на 14%..."`.

8. **`pdfs`** (`list` of `dict`):
   - Список словарей, содержащих информацию о скачанных PDF-файлах. Каждый словарь содержит:
     - **`name`** (`str`): Название PDF-файла.
       - Пример: `"it-market-analysis-2023.pdf"`.
     - **`content`** (`bytes`): Бинарное содержимое PDF-файла.

---

### Пример элемента списка `analytics`:
```json
{
    "link": "/news/12345",
    "date": "2023-10-15",
    "name": "Анализ рынка ИТ в 2023 году",
    "name_en": "it-market-analysis-2023",
    "tags": ["ИТ", "аналитика", "2023"],
    "pdf_links": [
        "https://b1.ru/local/assets/surveys/it-market-analysis-2023.pdf"
    ],
    "text": "В 2023 году рынок ИТ показал рост на 14%...",
    "pdfs": [
        {
            "name": "it-market-analysis-2023.pdf",
            "content": "binary_pdf_content_here"
        }
    ]
}
```

## b1-surveys
Парсим данные со страницы https://b1.ru/b1-surveys/

In [2]:
URL = 'https://b1.ru/'
SURVEYS_URL = URL + 'b1-surveys/'

session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from selenium.webdriver.chrome.options import Options
def get_chrome_options():
    # Настройки для подключения к удаленному Selenium
    options = Options()
    options.add_argument('--headless')  # Запуск в фоновом режиме
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=2560,1440')  # Принудительно устанавливает размер окна
    options.add_argument('--start-maximized')
    options.add_argument('--start-fullscreen')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument('--disable-gpu')  # Отключает аппаратное ускорение
    options.add_argument('--disable-infobars')  # Отключает информационные баннеры Chrome
    options.add_argument('--disable-features=TranslateUI')  # Отключает встроенный переводчик Chrome
    options.add_argument('--disable-popup-blocking')  # Отключает блокировку всплывающих окон
    options.add_argument('--disable-extensions')  # Отключает все расширения
    options.add_argument('--disable-notifications')  # Отключает уведомления
    options.add_argument('--disable-background-networking')  # Запрещает Chrome использовать сеть в фоне
    options.add_argument('--disable-sync')  # Отключает синхронизацию браузера
    options.add_argument('--disable-logging')  # Отключает логи браузера
    options.add_argument('--remote-debugging-port=9222')  # Включает отладку (для анализа поведения)
    options.add_argument('--force-device-scale-factor=1')  # Отключает автоадаптацию экрана
    options.add_argument('--disable-gesture-requirement-for-media-playback')  # Отключает автопроигрывание медиа

    # Прячем Selenium (чтобы сайт не распознавал бота)
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)

    # Устанавливаем user-agent браузера (чтобы эмулировать реальный Chrome)
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")
    return options


driver = webdriver.Remote(command_executor='http://chrome.dev.pp.ru:4444/wd/hub', options=get_chrome_options())

try:
    driver.get(SURVEYS_URL)
    wait = WebDriverWait(driver, 5)
    for _ in range(10):
        try:
            show_more_button = wait.until(EC.element_to_be_clickable((By.ID, "show-more")))
            driver.execute_script("arguments[0].click();", show_more_button)
            time.sleep(2)
        except:
            print("Кнопка 'Показать еще' больше не найдена или недоступна.")
            break

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
except Exception as e:
    print(f"Ошибка при ожидании элемента: {e}")
finally:
    driver.quit()

Кнопка 'Показать еще' больше не найдена или недоступна.


In [6]:
analytics = []
soup = BeautifulSoup(page_source, "html.parser")
for block in soup.find_all('div', class_=re.compile(r"^news-block__gridItem triangleWrap--hover")):
    if block:
        link = block['src-href']
        if not 'services' in link:
            analytics_block = {}
            analytics_block['link'] = link
            analytics_block['date'] = datetime.strptime(block.find('p', class_='news-block__date').text, '%d.%m.%Y')
            analytics_block['name'] = block.find('h2', class_='news-block__title').text
            analytics_block['name_en'] = link.strip().strip('/').split('/')[-1]
            
            time.sleep(1)
            analytics_soup = get_soup(URL + link)
            tags_block = analytics_soup.find('div', class_=re.compile(r"tezis-list-block__tags"))
            if tags_block:
                analytics_block['tags'] = [a.text for a in tags_block.find_all('a', class_=re.compile(r"tag"))]
            else:
                analytics_block['tags'] = []
            
            analytics_block['pdf_links'] = [f"{URL}{a['href']}" for a in analytics_soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
            analytics_block['pdf_links'].append(f'{URL}local/assets/surveys/{analytics_block["name_en"]}.pdf')

            main_text = analytics_soup.find('main')
            for el in main_text.find_all('section', attrs={"data-block-name": "Контакты"}):
                el.extract()
            for el in main_text.find_all('section', attrs={"data-block-name": "СКАЧАТЬ ПОЛНУЮ ВЕРСИЮ"}):
                el.extract()
            for el in main_text.find_all('div', attrs={"class": "page-cover"}):
                el.extract() 
            for el in main_text.find_all('script'):
                el.extract()
            for el in main_text.find_all('style'):
                el.extract()
            if tags_block:
                for el in tags_block:
                    el.extract()
            analytics_block['text'] = convert_to_text(main_text)
            
            pdfs_content = []
            pdfs_names = []
            for url in analytics_block['pdf_links']:
                for i in range(10):
                    try:
                        response = session.get(url, stream=True)
                        if response.status_code == 200:
                            pdf_name = url.strip().strip('/').split('/')[-1]
                            pdf_content = response.content
                            if pdf_name not in pdfs_names:
                                pdfs_names.append(pdf_name)
                                pdfs_content.append({"name": pdf_name, "content": pdf_content})
                            break
                    except:
                        time.sleep(3)
                    
            analytics_block['pdfs'] = pdfs_content
            analytics.append(analytics_block)

In [7]:
len(analytics)

58

In [8]:
with open('b1_analytics.pkl', 'wb') as f:
    pickle.dump(analytics, f)

## kamaflow
Парсим данные с сайта https://kamaflow.com/ru/blog/

In [5]:
URL = 'https://strategy.ru'

session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))

url = URL + "/api/v1/research/research"
params = {"limit": 10, "offset": 0}
response = session.get(url, params=params)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Ошибка: {response.status_code}")
    print(response.text)

researches_links = []
k = 10
for i in range(0, data['count'] + 1, k):
    params = {"limit": k, "offset": i}
    response = session.get(url, params=params)
    data = response.json()
    researches_links += data['researches']

In [6]:
researches = []
for research_block in tqdm(researches_links):
    link = URL + research_block['absolute_url']
    date = convert_date(research_block['published_at'])
    tags = [ind['title'] for ind in research_block['industries']]
    name = research_block['title']
    text_preview = research_block['preview']
    
    info = {'link': link, 'date': date, 'name': name, 'text_preview': text_preview, 'tags': tags}
    time.sleep(1)
    for i in range(10):
        try:
            soup = get_soup(info['link'])
            publication_div = soup.find('div', class_='publication__text tiny')
            pdf_links = []
            for btn in publication_div.find_all('a', class_='btn-download'):
                pdf_links.append(URL + btn.extract()['href'].replace('../', '/'))
            for el in publication_div.find_all('script'):
                    el.extract()
            for el in publication_div.find_all('style'):
                el.extract()
            info['text'] = convert_to_text(publication_div)
            info['pdf_links'] = list(set(pdf_links))
            pdfs_content = []
            for pdf_link in info['pdf_links']:
                response = session.get(pdf_link, stream=True)
                if response.status_code == 200:
                    pdf_name = url.strip().strip('/').split('/')[-1]
                    pdf_content = response.content
                    pdfs_content.append({"name": info['name'], "content": pdf_content})
            info['pdfs'] = pdfs_content
            break
        except:
            time.sleep(5)
    
    researches.append(info)
with open('kamaflow_researches.pkl', 'wb') as f:
    pickle.dump(researches, f)

  0%|          | 0/42 [00:00<?, ?it/s]

100%|██████████| 42/42 [01:27<00:00,  2.08s/it]


# Внутренние источники

Данные из аналитического хаба. Данные из  pdf  были получены с помощью PyPDFLoader.  
Дозаполним данные, чтобы структура данных была одинаковая

In [5]:
with open('industry_hub_analytics.pkl', 'rb') as f:
    industry_hub_analytics = pickle.load(f)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini")

In [71]:
prompt = PromptTemplate(
    input_variables=['name', "text"],
    template=(
        "Определи или извлеки наиболее вероятную дату создания документа. Докуменит представляет из себя отраслевая аналитика или отчёт на определённый период времени."
        "Если точный день неизвестен, укажи месяц и год. Если есть несколько дат, выбери наиболее вероятную. Учти, что даты на русском поэтому, скорее всего, если дата начианется с года, то потом идёт месяц, либо сначала идёт день, а потом месяц."
        "Выведи только дату в формате YYYY-MM-DD."
        "Наименование самого документа: :\n\n{name}\n\n"
        "Текст документа (извлёчен из pdf), :\n\n{text}\n\n"
        
    )
)

industry_hub_analytics_update = []

for hub in tqdm(industry_hub_analytics):
    for pdf_block in hub['pdfs']:
        response = llm.invoke(prompt.format(name=pdf_block['name'], text=pdf_block['full_text'])[:1_500])
    
        data = {
            "link": hub['link'],
            "date": response.content,
            "name": pdf_block['name'],
            "tags": hub['tags'],
            "text": "",
            "pdfs": [pdf_block]
        }
        industry_hub_analytics_update.append(data)
        
with open('industry_hub_analytics_update.pkl', 'wb') as f:
    pickle.dump(industry_hub_analytics_update, f)

100%|██████████| 16/16 [02:22<00:00,  8.90s/it]


# 